In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import csv
import os

In [ ]:
# MOB subtractor
subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

In [ ]:
# kp and des methods

sift = cv2.xfeatures2d.SIFT_create()
def siftKpAndDes(frame):
    return sift.detectAndCompute(frame,None)

surf = cv2.xfeatures2d.SURF_create()
def surfKpAndDes(frame):
    return surf.detectAndCompute(frame,None)

star = cv2.xfeatures2d.StarDetector_create()
def fastKp(frame):
    return star.detect(frame,None)

brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
def briefDes(frame, kps):
    return brief.compute(frame, kps)

orb = cv2.ORB_create()
def orbKpAndDes(frame):
    return orb.detectAndCompute(frame,None)

In [ ]:
# matcher methods

FLANN_INDEX_KDTREE = 1
FLANN_KDTREES = 5

FLANN_INDEX_LSH = 6
TABLE_NUMBER = 6 #12
KEY_SIZE = 12 #20
MULTI_PROBE_LEVEL = 1 #2
SEARCH_CHECKS = 50

#BF matching
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING)
bf_sift = cv2.BFMatcher()
def bfMatcher(des1, des2, feature_type = 'SIFT', ratio = 0.7):

    if feature_type is 'ORB': 
        matches = bf_orb.knnMatch(des1,des2,k=2)
    else:
        matches = bf_sift.knnMatch(des1, des2, k=2)
    # filter using ratio
    good = []
    for pair in matches:
        try:
            m,n = pair
            if m.distance < ratio*n.distance:
                good.append(m)
        except ValueError:
            print('Missing Match pair!')
            passq
    return good, matches


#FLANN-based
search_params = dict(checks = SEARCH_CHECKS)
flann_orb = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_LSH,
                   table_number = TABLE_NUMBER,
                   key_size = KEY_SIZE,
                   multi_probe_level = MULTI_PROBE_LEVEL),
                   search_params)
flann_sift = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_KDTREE, trees = FLANN_KDTREES), 
                                   search_params )
def flannMatcher(des1, des2, feature_type = 'SIFT', ratio=0.7):
    
    if feature_type is 'ORB': 
        matches = flann_orb.knnMatch(des1, des2, k=2)
    else:
        matches = flann_sift.knnMatch(des1, des2, k=2)
    # filter using ratio
    good = []
    for pair in matches:
        try:
            m,n = pair
            if m.distance < ratio*n.distance:
                good.append(m)
        except ValueError:
            print('Missing Match pair!')
            pass
    return good, matches
        

In [ ]:
# preprocess kp/des for query image(s)

img1 = cv2.imread('logos/rolex.jpg',0)          # queryImage
# find the keypoints and descriptors
kp1, des1 = siftKpAndDes(img1)

img1kps = cv2.drawKeypoints(img1, kp1, None, color=(0,255,0), flags=0)
plt.figure(figsize=(20,20))
plt.imshow(img1kps), plt.show()

print(len(kp1))

In [ ]:
# encoding params for quality when saving sample jpeg screenshots
encoding_params = [int(cv2.IMWRITE_JPEG_QUALITY), 10]

def runMatching(query, video, output_dir="outputs", chain = 3, feature_ext = 'ORB', matcher = 'FLANN',
               frame_interval = 1, ratio = 0.7, min_matches = 15):
    print(feature_ext)
    print(matcher)
    positives = [] # list of frame numbers of positive indentifications
    n_pos = 0
    errors = [] # list of error frames to analyze later (hopefully should be empty!)
    
    cap = cv2.VideoCapture(video)
    img_chain = [] # list of past images in the current running chain of positive frames, up to the chain parameter
    
    # preprocess kp/des for query image(s)

    img1 = cv2.imread(query,0) # queryImage
    # find the keypoints and descriptors
    kp1, des1 = orbKpAndDes(img1)
    
    while(True):
        for i in range(frame_interval):
            #skip i number of frames
            cap.grab()

        # Capture frame-by-frame and covert to grayscale
        ret, frame = cap.read()
        if not ret:
            print("Analysis finished!")
            #TODO: post-processing of CSV and image files
            with open(os.path.join(output_dir,'results.csv'), 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['SUMMARY OF RESULTS'])
                writer.writerow(['Query Image', 'Video', 'Chain Length', 'Feature Extractor', 'Matcher', 
                                 'Frame Interval', 'Pass Ratio', 'Match Thresh'])
                write.writerow([query, video, chain, feature_ext, matcher, frame_interval, ratio, min_matches])
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #     # TESTING: MOG2 background subtractor mask
    #     mask = subtractor.apply(gray, masked, 0.999)
    #     masked = cv2.bitwise_and(gray, masked)
    #     kp2, des2 = siftKpAndDes(masked)


    #     # TESTING: Equalize historgram to incrase contrast (probably for ORB)
    #     eh = cv2.equalizeHist(gray)
    #     kp2, des2 = siftKpAndDes(eh)

    #     # TESTING: Blurring with various LPFs
    #     blurred = cv2.bilateralFilter(gray,9,75,75)
    #     kp2, des2 = siftKpAndDes(blurred)

        # DEFAULT: find the keypoints and descriptors
        kp2, des2 = orbKpAndDes(gray)

    #     # DEBUGGING: show only target image kps
    #     img3 = cv2.drawKeypoints(blurred, kp2, None, color=(0,255,0), flags=0)
    #     cv2.imshow('frame',img3)
    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break
    #     continue
        try:
            if matcher is 'FLANN':     
                good, matches = flannMatcher(des1, des2, feature_type=feature_ext, ratio=ratio)
            else: # else we use BF
                good, matches = bfMatcher(des1, des2, feature_type=feature_ext, ratio=ratio)

            if len(good) > min_matches:
                print(len(good))
                src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

                if M is None:
                    # if homography matrix is empty
                    print("Empty homography matrix")
                    img3 = gray # display image without any lines
                else:
        #             print(M)
                    matchesMask = mask.ravel().tolist()
                    h,w = img1.shape
                    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                    dst = cv2.perspectiveTransform(pts,M)

                    # covert to color so homography line can be shown
                    color = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
                    img3 = cv2.polylines(color,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)  
            else:
                print( "Not enough matches are found - {}/{}".format(len(good), min_matches) )
                matchesMask = None
                img3 = gray # display image without any lines

        #     draw_params = dict(matchColor = (0,255,0), # draw matches in green color
        #                        singlePointColor = None,
        #                        matchesMask = matchesMask, # draw only inliers
        #                        flags = 2)
        #     img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

        #     # Display the resulting frame
        #     cv2.imshow('frame',gray)
            print(chain)
            print(len(img_chain))
            
        except cv2.error as e:
            print('Matching Error! Recording and skipping this frame...')
            errors.append(e)
            pass

        if len(good) > min_matches and M is not None:
            img_chain.append(img3) # add this image to the current continuous chain of positive frames
            if len(img_chain) is chain:
                for img_in_chain in img_chain:
                    n_pos += 1
                    img_name = 'frame%d.jpg' % n_pos
                    cv2.imwrite(os.path.join(output_dir, img_name), img3, encoding_params) # write with compression param as specified up top
                    print(img_name)
            elif len(img_chain) > chain:
                n_pos += 1
                img_name = 'frame%d.jpg' % n_pos
                cv2.imwrite(os.path.join(output_dir, img_name), img3, encoding_params) # write with compression param as specified up top
                print(img_name)
        else:
            img_chain = [] # if negative, then clear chain
            
        cv2.putText(img3, "T.Count:" + str(n_pos), (2,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(img3, "Pos.Chain:" + str(len(img_chain)), (2,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow('frame',img3)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    return positives, n_pos, errors

In [ ]:
# runner
# TODO: make all params specified here!

FRAME_INTERVAL = 1
MIN_MATCH_COUNT =  15 #RULE: >10% of keypoints in query image? -> work well only fort scale-invarient (SIFT)
RATIO = 0.7 #lowered from 0.7 to derease sensitivity

runMatching('logos/rolex.jpg', 'videos/15brazil-5min.mp4', output_dir='outputs', chain=3,
            feature_ext = 'ORB', matcher='FLANN',
            frame_interval=FRAME_INTERVAL, ratio=RATIO, min_matches=MIN_MATCH_COUNT)
